- Cosmos DB is not a single database engine — it provides multiple APIs over a distributed storage + indexing engine:
    - SQL API (Core API) → JSON document store with SQL-like queries.
    - MongoDB API → Emulates MongoDB wire protocol.
    - Cassandra API → Wide column store.
    - Gremlin API → Graph database queries.
    - Table API → Key-value store.
- 

In [ ]:
from openai import OpenAI
import json
import os
from azure.cosmos import CosmosClient, exceptions, PartitionKey
from azure.cosmos.exceptions import CosmosResourceNotFoundError
from typing import Dict, Any, List
from langchain_community.document_loaders import PDFPlumberLoader
import uuid

from config import load_config_data

end_point = load_config_data["AZURE_COSMOS_DB_URL"]
api_key = load_config_data["AZURE_COSMOS_DB_PRIMARY_KEY"]


embedding_base_url = load_config_data["MISTRAL_BASE_URL"]
embedding_api_key = load_config_data["MISTRAL_API_KEY"]
embedding_model = load_config_data["MISTRAL_EMBEDDING"]

DATABASE_NAME = "Test-db"
CONTAINER_NAME = "Test_1"
KNOWLEDGE_INDEX_NAME = "Test_index_1"
PARTITION_KEY = "/vector_id"

cosmos_client = CosmosClient(end_point, credential=api_key)

llm_client = OpenAI(api_key=embedding_api_key, base_url=embedding_base_url)

In [ ]:
def generate_embedding(text: str) -> List[float]:
    """
    Returns the embedding vector for the given text.
    
    Parameters:
        text (str): The input text to embed.
            
    Returns:
        list[float]: The embedding vector.
    """
    response = llm_client.embeddings.create(model=embedding_model, input=[text])
    
    return response.data[0].embedding

In [ ]:
def get_or_create_cosmos_container(
    database_name: str, 
    container_name: str, 
    partition_key_path: str = "/id",
    vector_path: str = "/vector",
    dimensions: int = 1024,
    distance_function: str = "cosine"
):
    """
    Checks for the existence of a Cosmos DB database and container, creating them if necessary.
    Includes the required vector indexing policy for RAG/chatbot applications.

    Args:
        database_name (str): The name of the database to connect to or create.
        container_name (str): The name of the container (collection) to connect to or create.
        partition_key_path (str): The path for the container's partition key (e.g., '/vector_id').
        vector_path (str): The path to the vector field in your documents (e.g., '/vector').
        dimensions (int): The dimension size of your vector embeddings (e.g., 1024).
        distance_function (str): The distance metric to use (e.g., 'cosine', 'dot_product', 'euclidean').

    Returns:
        azure.cosmos.container.ContainerProxy: The client object for the specified container.
    """
    try:
        database = cosmos_client.create_database_if_not_exists(id=database_name)
        print(f"⚙️ Database '{database_name}' is ready.")

        vector_policy_definition = {
            "vectorEmbeddings": [
                {
                    "path": vector_path,
                    "dataType": "float32",
                    "distanceFunction": distance_function,
                    "dimensions": dimensions
                }
            ]
        }

        indexing_policy_definition = {
            "indexingMode": "consistent",
            "automatic": True,
            "includedPaths": [{"path": "/*"}],
            "excludedPaths": [{"path": '/"_etag"/?'}],
            "vectorIndexes": [{
                "id": "vector_index_01",
                "path": vector_path, 
                "type": "diskANN" 
            }]
        }

        container = database.create_container_if_not_exists(id=container_name,
                                                            partition_key=PartitionKey(path=partition_key_path),
                                                            indexing_policy=indexing_policy_definition,
                                                            vector_embedding_policy=vector_policy_definition,
                                                            offer_throughput=400)
        print(f"⚙️ Container '{container_name}' (Partition Key: {partition_key_path}) is ready.")
        return container

    except Exception as e:
        print(f"❌ An error occurred during Cosmos DB setup: {e}")
        return None

In [ ]:
containor_client = get_or_create_cosmos_container(DATABASE_NAME, CONTAINER_NAME)

In [ ]:
def file_page_data(file_path):
    data_to_upload = []
    if os.path.basename(file_path).endswith(".pdf"):
        loader = PDFPlumberLoader(file_path)
        data = loader.load_and_split()            
        for item in data:
            data_to_upload.append({"id":str(uuid.uuid4()),
                                "knowledge_id": "Test_1",
                                "content":item.page_content,
                                "vector": generate_embedding(item.page_content),
                                "metadata" : {
                                    "source": item.metadata['source'],
                                    "file_path": item.metadata['file_path'],
                                    "page" : int(item.metadata['page'])+1}})
    elif os.path.basename(file_path).endswith(".docx"):
        pass
    else:
        return "Something went wrong"
    return data_to_upload

In [ ]:
def batch_upload_documents(
    container_client, 
    documents_to_upload: list[dict],
    vector_field_name: str,
    partition_key_name: str
):
    """
    Uploads or updates a batch of documents in the specified Cosmos DB container using upsert_item.

    Args:
        container_client (azure.cosmos.container.ContainerProxy): The client object for the container.
        documents_to_upload (list[dict]): A list of pre-structured documents to upsert.
        vector_field_name (str): The name of the field where the vector embedding should be stored (e.g., 'vector').
        partition_key_name (str): The name of the field used as the partition key (e.g., 'vector_id').
    """
    uploaded_count = 0
    failed_count = 0
    
    vector_field_name_clean = vector_field_name.strip("/")
    partition_key_name_clean = partition_key_name.strip("/")

    for item in documents_to_upload:
        try:
            if vector_field_name_clean not in item:
                raise ValueError(f"Document missing required vector field '{vector_field_name_clean}'.")

            # 2. Ensure the partition key field exists (use 'id' as fallback if partition key is missing)
            if partition_key_name_clean not in item and "id" in item:
                # Create the partition key using the item's 'id' field
                item[partition_key_name_clean] = f"vec-{item['id']}"
            
            # 3. Final check for partition key value
            if partition_key_name_clean not in item:
                raise ValueError(f"Document missing required partition key '{partition_key_name_clean}' or 'id'.")

            partition_key_value = item[partition_key_name_clean]

            # 4. Perform the upsert operation (insert or replace)
            container_client.upsert_item(body=item)
            uploaded_count += 1
            
        except Exception as e:
            failed_count += 1
            doc_id = item.get("id", "N/A")
            print(f"❌ Failed to upsert document chunk '{doc_id}': {e}")
    
    print(f"\n✨ Batch Upload Complete: {uploaded_count} documents uploaded, {failed_count} failed.")

In [ ]:
def final_document_upload_cosmos_db(file_path, db_name, container_name):
    container_client = get_or_create_cosmos_container(db_name, container_name)
    data = file_page_data(file_path)
    print("Uploading documents...")
    batch_upload_documents(container_client, data, "/vector", PARTITION_KEY)
    return "✅ Successfuly uploaded to cosmos DB"

In [ ]:
file_path = r"C:\Users\Ambik\Downloads\Machine-Learning-Interview-Questions-.pdf"
final_document_upload_cosmos_db(file_path, DATABASE_NAME, CONTAINER_NAME)

In [ ]:
def vector_search_documents(
    container_client, 
    user_query: str, 
    vector_path: str = "/vector",
    distance_function: str = "cosine",
    top_k: int = 3
) -> list[dict]:
    """
    Performs a vector similarity search against the Cosmos DB container.

    Args:
        container_client (azure.cosmos.container.ContainerProxy): The client object for the container.
        user_query (str): The text query from the user.
        dimensions (int): The dimension size of the vector embeddings.
        vector_path (str): The path to the vector field in the documents (e.g., '/vector').
        distance_function (str): The distance metric to use (e.g., 'cosine').
        top_k (int): The number of top matching documents to retrieve.

    Returns:
        list[dict]: A list of relevant documents with their similarity scores.
    """
    print(f"\n--- Searching for top {top_k} documents for query: '{user_query}' ---")
    
    # 1. Generate the vector embedding for the user query
    query_vector = generate_embedding(user_query)
    
    # Clean the vector path for use in the SQL query (e.g., "/vector" -> "vector")
    vector_field_name_clean = vector_path.strip("/")

    sql_query = f"""
        SELECT TOP {top_k} 
            c.id, 
            c.content, 
            c.metadata,
            VectorDistance(c.{vector_field_name_clean}, @queryVector, false) AS similarity 
        FROM c 
        ORDER BY VectorDistance(c.{vector_field_name_clean}, @queryVector, false)
    """

    parameters = [{"name": "@queryVector", "value": query_vector}]

    try:
        results = container_client.query_items(
            query=sql_query,
            parameters=parameters,
            enable_cross_partition_query=True
        )

        relevant_documents = list(results)
        print(f"✅ Successfully retrieved {len(relevant_documents)} relevant documents.")
        return relevant_documents

    except Exception as e:
        print(f"❌ An error occurred during vector search: {e}")
        return []

In [ ]:
query = " What's the trade-off between bias and variance?"
vector_search_documents(containor_client, query)

In [ ]:
def delete_documents_by_metadata(
    container_client,
    filter_key: str,
    filter_value: str
):
    """
    Deletes all documents that match a specific metadata filter (key and value).
    
    Args:
        container_client (azure.cosmos.container.ContainerProxy): The client object for the container.
        filter_key (str): The document field name to filter on (e.g., 'knowledge_id' or 'metadata.source').
        filter_value (str): The exact value to match for deletion.
    """
    filter_key = f"metadata.{filter_key}"
    print(f"\n--- Starting deletion for documents where {filter_key} = '{filter_value}' ---")
    
    deleted_count = 0
    
    try:
        # 1. Retrieve the container properties to get the partition key path
        properties = container_client.read()
        partition_key_path = properties.get('partitionKey', {}).get('paths', [''])[0].strip('/')
        
        if not partition_key_path:
            raise ValueError("Could not determine the container's partition key path.")

        # 2. Build the query to select only the required fields for deletion
        # FIX: Alias the partition key field to 'pk_value' to avoid the conflict 
        # that results in the "property name 'id' specified more than once" error.
        sql_query = f"""
            SELECT c.id, c.{partition_key_path} AS pk_value
            FROM c 
            WHERE c.{filter_key} = @filterValue
        """
        
        # 3. Define the parameters for the query
        parameters = [
            {"name": "@filterValue", "value": filter_value}
        ]
        
        # 4. Query to find the documents to delete
        results = container_client.query_items(
            query=sql_query,
            parameters=parameters,
            enable_cross_partition_query=True
        )

        # 5. Iterate over results and delete each document
        for doc in results:
            doc_id = doc.get("id")
            # Get the partition key value using the alias 'pk_value'
            pk_value = doc.get('pk_value') 
            
            if doc_id and pk_value:
                # The delete operation requires both the document ID and the partition key value.
                container_client.delete_item(
                    item=doc_id, 
                    partition_key=pk_value
                )
                deleted_count += 1
            
        print(f"✅ Deletion complete. Total documents deleted: {deleted_count}")
        return deleted_count

    except Exception as e:
        print(f"❌ An error occurred during metadata-based deletion: {e}")
        return 0

In [ ]:
delete_documents_by_metadata(containor_client, 'source', "C:\\Users\\Ambik\\Downloads\\Machine-Learning-Interview-Questions-.pdf")

In [ ]:
def delete_cosmos_container(database_name: str, container_name: str):
    """
    Deletes a specified container from the Cosmos DB database.

    Args:
        database_name (str): The name of the database containing the container.
        container_name (str): The name of the container to be deleted.
    """
    try:
        database = cosmos_client.get_database_client(database_name)
        
        # 3. Get the container reference and delete it
        database.delete_container(container_name)
        print(f"\n🗑️ Successfully deleted container '{container_name}' from database '{database_name}'.")

    except CosmosResourceNotFoundError:
        print(f"\n⚠️ Container '{container_name}' or Database '{database_name}' not found. Deletion skipped.")
    except Exception as e:
        print(f"\n❌ An error occurred during container deletion: {e}")

In [ ]:
langchain langchain-core langchain-community langchain-openai langchain-huggingface langgraph llama-index-core llama-index-llms-openai llama-index-llms-replicate llama-index-embeddings-huggingface